In [1]:
#imports
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [2]:
#read files
import zipfile
with zipfile.ZipFile('customers.csv.zip', 'r') as z:
    f = z.open('customers.csv')
    customers = pd.read_csv(f)
    
with zipfile.ZipFile('articles.csv.zip', 'r') as z:
    f = z.open('articles.csv')
    articles = pd.read_csv(f)
    
with zipfile.ZipFile('transactions_train.csv.zip', 'r') as z:
    f = z.open('transactions_train.csv')
    transactions = pd.read_csv(f, parse_dates=['t_dat'])

In [3]:
def select_columns(df):
    return (df
            .iloc[:,:-1]
           )

def tweak_data(df):
    return (df
     .pipe(select_columns)
     .assign(FN = df.FN.fillna(0).astype(bool),
             Active = df.Active.fillna(0).astype(bool),
             fashion_news_frequency = lambda df: df.fashion_news_frequency.where(df.fashion_news_frequency != 'None', 'NONE').astype('category')
             )
     .dropna()
     .assign(club_member_status = df.club_member_status.astype('category'),
            age = df.age.astype('Int8'))
           )

In [4]:
tweak_data(customers).info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1338570 entries, 0 to 1371979
Data columns (total 6 columns):
 #   Column                  Non-Null Count    Dtype   
---  ------                  --------------    -----   
 0   customer_id             1338570 non-null  object  
 1   FN                      1338570 non-null  bool    
 2   Active                  1338570 non-null  bool    
 3   club_member_status      1338570 non-null  category
 4   fashion_news_frequency  1338570 non-null  category
 5   age                     1338570 non-null  Int8    
dtypes: Int8(1), bool(2), category(2), object(1)
memory usage: 28.1+ MB


In [5]:
transactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31788324 entries, 0 to 31788323
Data columns (total 5 columns):
 #   Column            Dtype         
---  ------            -----         
 0   t_dat             datetime64[ns]
 1   customer_id       object        
 2   article_id        int64         
 3   price             float64       
 4   sales_channel_id  int64         
dtypes: datetime64[ns](1), float64(1), int64(2), object(1)
memory usage: 1.2+ GB


In [6]:
np.finfo(np.float32)

finfo(resolution=1e-06, min=-3.4028235e+38, max=3.4028235e+38, dtype=float32)

In [7]:
transactions.describe()

,article_id,price,sales_channel_id
count,3.178832e+07,3.178832e+07,3.178832e+07
mean,6.962272e+08,2.782927e-02,1.704028e+00
std,1.334480e+08,1.918113e-02,4.564786e-01
min,1.087750e+08,1.694915e-05,1.000000e+00
25%,6.328030e+08,1.581356e-02,1.000000e+00
50%,7.145820e+08,2.540678e-02,2.000000e+00
75%,7.865240e+08,3.388136e-02,2.000000e+00
max,9.562170e+08,5.915254e-01,2.000000e+00


In [8]:
def tweak_data_trns(df):
    return (df
     .assign(price = df.price.astype('float32'),
             sales_channel = 'store')
     .assign(sales_channel = lambda df: df.sales_channel.where(df.sales_channel_id == 1, 'online').astype('category')
             )
     .drop(columns=['sales_channel_id'])
           )

In [9]:
tweak_data_trns(transactions).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31788324 entries, 0 to 31788323
Data columns (total 5 columns):
 #   Column         Dtype         
---  ------         -----         
 0   t_dat          datetime64[ns]
 1   customer_id    object        
 2   article_id     int64         
 3   price          float32       
 4   sales_channel  category      
dtypes: category(1), datetime64[ns](1), float32(1), int64(1), object(1)
memory usage: 879.2+ MB


In [10]:
def tweak_data_arts(df):
    cols = ['article_id','section_name']
    return (df
     [cols]
     .assign(section_name = df.section_name.astype('category'),
             )
     )

In [11]:
tweak_data_arts(articles).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105542 entries, 0 to 105541
Data columns (total 2 columns):
 #   Column        Non-Null Count   Dtype   
---  ------        --------------   -----   
 0   article_id    105542 non-null  int64   
 1   section_name  105542 non-null  category
dtypes: category(1), int64(1)
memory usage: 930.7 KB


In [12]:
def merge_dfs(customers, transactions, articles):
    return (
    pd.merge(pd.merge(tweak_data(customers), tweak_data_trns(transactions), on='customer_id'),
            tweak_data_arts(articles), on='article_id')
    )

In [13]:
df = (merge_dfs(customers, transactions, articles)
            .drop(columns=['article_id', 't_dat']))

In [14]:
filter_ = ((merge_dfs(customers, transactions, articles)['customer_id'].value_counts()>1)&
           (merge_dfs(customers, transactions, articles)['customer_id'].value_counts()<=13))

filter_ = (merge_dfs(customers, transactions, articles)
            .drop(columns=['article_id', 't_dat'])
            ['customer_id'].isin(
                merge_dfs(customers, transactions, articles)['customer_id'].value_counts()[filter_].index)
           )

filter_2 = ((merge_dfs(customers, transactions, articles)['customer_id'].value_counts()>13)&
           (merge_dfs(customers, transactions, articles)['customer_id'].value_counts()<=30))

filter_2 = (merge_dfs(customers, transactions, articles)
            .drop(columns=['article_id', 't_dat'])
            ['customer_id'].isin(
                merge_dfs(customers, transactions, articles)['customer_id'].value_counts()[filter_2].index)
           )

filter_3 = (merge_dfs(customers, transactions, articles)['customer_id'].value_counts()>30)
           

filter_3 = (merge_dfs(customers, transactions, articles)
            .drop(columns=['article_id', 't_dat'])
            ['customer_id'].isin(
                merge_dfs(customers, transactions, articles)['customer_id'].value_counts()[filter_3].index)
           )

df.loc[filter_, 'purchaser_type'] = 'low'
df.loc[filter_2, 'purchaser_type'] = 'medium'
df.loc[filter_3, 'purchaser_type'] = 'heavy'
df = df.dropna()
df = df.assign(purchaser_type = df.purchaser_type.astype('category'))

In [15]:
df['price'].describe()

count    3.134623e+07
mean     2.591957e-02
std      1.872030e-02
min      1.694915e-05
25%      1.577966e-02
50%      2.540678e-02
75%      3.388136e-02
max      5.915254e-01
Name: price, dtype: float64

In [16]:
df_by_sales_channel = (df.groupby(['customer_id', 'sales_channel']).agg({'FN':'count'})
 .pivot_table(index=['customer_id'], columns=['sales_channel'], values=['FN'])
['FN'])
df_by_sales_channel.columns = pd.Index(list(df_by_sales_channel.columns))
df_by_sales_channel

,online,store
customer_id,,
00000dbacae5abe5e23885899a1fa44253a17956c6d1c3d25f88aa139fdfc657,12,9
0000423b00ade91418cceaf3b26c6af3dd342b51fd051eec9c12fb36984420fa,81,5
000058a12d5b43e67d225668fa1f8d618c13dc232df0cad8ffe7ad4a1091e318,18,0
00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2c5feb1ca5dff07c43e,2,0
00006413d8573cd20ed7128e53b7b13819fe5cfc2d801fe7fc0f26dd8d65a85a,11,2
...,...,...
ffffa28cd7ab5d1cbbbfe7b582b1c419270cc0539f3dae5bdfa4cf4b5874c806,0,3
ffffbbf78b6eaac697a8a5dfbfd2bfa8113ee5b403e4747568cac33e8c541831,38,13
ffffcd5046a6143d29a04fb8c424ce494a76e5cdf4fab53481233731b5c4f8b7,56,28


In [17]:
df_by_sales_channel = (df_by_sales_channel
 .assign(all_online = np.where(df_by_sales_channel.store==0, 1, 0).astype('bool'),
        all_store = np.where(df_by_sales_channel.online==0, 1, 0).astype('bool'),
        total = (df_by_sales_channel['online'] + df_by_sales_channel['store']).astype('Int16'),
        avg_price = df.groupby('customer_id').agg({'price':'mean'}).values)
 .drop(columns=['online', 'store'])
                      )

In [18]:
#np.iinfo(np.int16)

In [19]:
df_by_sales_channel

,all_online,all_store,total,avg_price
customer_id,,,,
00000dbacae5abe5e23885899a1fa44253a17956c6d1c3d25f88aa139fdfc657,False,False,21,0.030904
0000423b00ade91418cceaf3b26c6af3dd342b51fd051eec9c12fb36984420fa,False,False,86,0.030255
000058a12d5b43e67d225668fa1f8d618c13dc232df0cad8ffe7ad4a1091e318,True,False,18,0.039154
00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2c5feb1ca5dff07c43e,True,False,2,0.030492
00006413d8573cd20ed7128e53b7b13819fe5cfc2d801fe7fc0f26dd8d65a85a,False,False,13,0.036130
...,...,...,...,...
ffffa28cd7ab5d1cbbbfe7b582b1c419270cc0539f3dae5bdfa4cf4b5874c806,False,True,3,0.029367
ffffbbf78b6eaac697a8a5dfbfd2bfa8113ee5b403e4747568cac33e8c541831,False,False,51,0.025340
ffffcd5046a6143d29a04fb8c424ce494a76e5cdf4fab53481233731b5c4f8b7,False,False,84,0.021516


In [20]:
df_new = (df_by_sales_channel
 .merge(df[['customer_id', 'FN', 'Active', 'club_member_status', 'fashion_news_frequency', 'age', 'purchaser_type']],
       how='left', left_index=True, right_on='customer_id')
 .drop_duplicates()
)

df_new

,all_online,all_store,total,avg_price,customer_id,FN,Active,club_member_status,fashion_news_frequency,age,purchaser_type
0,False,False,21,0.030904,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,False,False,ACTIVE,NONE,49,medium
43462,False,False,86,0.030255,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,False,False,ACTIVE,NONE,25,heavy
72033,True,False,18,0.039154,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,False,False,ACTIVE,NONE,24,medium
245535,True,False,2,0.030492,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,False,False,ACTIVE,NONE,54,low
249281,False,False,13,0.036130,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,True,True,ACTIVE,Regularly,52,low
...,...,...,...,...,...,...,...,...,...,...,...
2322805,False,True,3,0.029367,ffffa28cd7ab5d1cbbbfe7b582b1c419270cc0539f3dae...,True,True,ACTIVE,Regularly,22,low
944302,False,False,51,0.025340,ffffbbf78b6eaac697a8a5dfbfd2bfa8113ee5b403e474...,False,False,ACTIVE,NONE,24,heavy
212747,False,False,84,0.021516,ffffcd5046a6143d29a04fb8c424ce494a76e5cdf4fab5...,False,False,ACTIVE,NONE,21,heavy
212748,False,False,45,0.017532,ffffcf35913a0bee60e8741cb2b4e78b8a98ee5ff2e6a1...,True,True,ACTIVE,Regularly,21,heavy


In [27]:
df_new = df_new.set_index('customer_id')

In [52]:
df_new = (df_new
 .assign(price_trf = np.round(df_new.avg_price * 1000000).astype('int32'))
 .drop(columns='avg_price')
)

In [53]:
#np.iinfo(np.int32)

In [69]:
df_new = (df_new
 .drop(columns='total')
)

In [73]:
df_new = (df_new
 .drop(columns=['FN','Active'])
)

In [74]:
df_new.dtypes

all_online                    bool
all_store                     bool
club_member_status        category
fashion_news_frequency    category
age                           Int8
purchaser_type            category
price_trf                    int32
dtype: object

In [75]:
df_new.iloc[:10,:]

,all_online,all_store,club_member_status,fashion_news_frequency,age,purchaser_type,price_trf
customer_id,,,,,,,
00000dbacae5abe5e23885899a1fa44253a17956c6d1c3d25f88aa139fdfc657,False,False,ACTIVE,NONE,49,medium,30904
0000423b00ade91418cceaf3b26c6af3dd342b51fd051eec9c12fb36984420fa,False,False,ACTIVE,NONE,25,heavy,30255
000058a12d5b43e67d225668fa1f8d618c13dc232df0cad8ffe7ad4a1091e318,True,False,ACTIVE,NONE,24,medium,39154
00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2c5feb1ca5dff07c43e,True,False,ACTIVE,NONE,54,low,30492
00006413d8573cd20ed7128e53b7b13819fe5cfc2d801fe7fc0f26dd8d65a85a,False,False,ACTIVE,Regularly,52,low,36130
0000757967448a6cb83efb3ea7a3fb9d418ac7adf2379d8cd0c725276a467a2a,True,False,ACTIVE,NONE,20,low,27667
00007d2de826758b65a93dd24ce629ed66842531df6699338c5570910a014cc2,False,False,ACTIVE,Regularly,32,heavy,31863
00007e8d4e54114b5b2a9b51586325a8d0fa74ea23ef77334eaec4ffccd7ebcc,False,True,ACTIVE,NONE,20,low,26678
00008469a21b50b3d147c97135e25b4201a8c58997f78782a0cc706645e14493,False,True,ACTIVE,NONE,20,low,19517


In [76]:
gower.gower_topn(df_new.iloc[:1,:],df_new.iloc[:10,:], cat_features=[True,True,True,True,False,True,False], n=5)

{'index': array([0, 1, 2, 3, 4], dtype=int64),
 'values': array([0.        , 0.24841888, 0.30791706, 0.30971995, 0.33633792],
       dtype=float32)}

In [77]:
#!pip install gower
import gower

distance_matrix = gower.gower_matrix(df_new, cat_features=[True,True,True,True,False,True,False])

MemoryError: Unable to allocate 5.27 TiB for an array with shape (1203649, 1203649) and data type float32

In [15]:
(merge_dfs(customers, transactions, articles)
            .drop(columns=['article_id', 't_dat'])
               [filter_])

,customer_id,FN,Active,club_member_status,fashion_news_frequency,age,price,sales_channel,section_name


In [15]:
df_filtered = (merge_dfs(customers, transactions, articles)
            .drop(columns=['article_id', 't_dat'])
               [filter_])
df_filtered

,customer_id,FN,Active,club_member_status,fashion_news_frequency,age,price,sales_channel,section_name
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,False,False,ACTIVE,NONE,49,0.044051,store,Kids Outerwear
2,05d20eff33ec820d4330ee25cce4d0c35e8a305137caf0...,True,True,ACTIVE,Regularly,41,0.033881,online,Kids Outerwear
3,05fa492d5d182adb9dca7cc20be545d2dc8990606d2197...,True,False,ACTIVE,Regularly,41,0.050831,store,Kids Outerwear
4,060de711fa176fb5249bb39f0eceea040bde903f8b0434...,False,False,ACTIVE,NONE,45,0.067780,online,Kids Outerwear
5,082bdebef72e43616dfd606c8665bf35321898afcf640c...,False,False,ACTIVE,NONE,25,0.027102,online,Kids Outerwear
...,...,...,...,...,...,...,...,...,...
31471567,ffe6e53e0ac98cd3da1697e057f24f002482ef00d0573b...,True,True,ACTIVE,Regularly,22,0.008458,store,Divided Accessories
31471568,fff22c025716b0f38fadb92a2bd09881c98e11368589c1...,True,True,ACTIVE,Regularly,41,0.020322,online,Womens Everyday Basics
31471569,fff3e75605ec575be9b95eda1e6557299e81bba12668d7...,True,True,ACTIVE,Regularly,33,0.023712,online,Kids Girl
31471570,fff4381593e170ca0aea188998487c830d9a4070c9ec4b...,False,False,ACTIVE,NONE,27,0.015237,online,Ladies H&M Sport


In [16]:
df_filtered.info(verbose=False, memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31346234 entries, 0 to 31471571
Columns: 9 entries, customer_id to section_name
dtypes: Int8(1), bool(2), category(4), float32(1), object(1)
memory usage: 4.1 GB


In [17]:
df_filtered[['customer_id', 'section_name', 'FN']].groupby(['customer_id', 'section_name']).count()

MemoryError: 

In [17]:
merge_dfs(customers, transactions, articles).drop(columns=['article_id', 't_dat'])[['section_name', 'FN']].reset_index().groupby(['customer_id', 'section_name']).count()

KeyError: 'customer_id'

In [8]:
merge_dfs(customers, transactions, articles).drop(columns=['article_id']).info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31471572 entries, 0 to 31471571
Data columns (total 11 columns):
 #   Column                  Dtype         
---  ------                  -----         
 0   customer_id             object        
 1   FN                      bool          
 2   Active                  bool          
 3   club_member_status      category      
 4   fashion_news_frequency  category      
 5   age                     Int8          
 6   t_dat                   datetime64[ns]
 7   article_id              int64         
 8   price                   float32       
 9   sales_channel           category      
 10  section_name            category      
dtypes: Int8(1), bool(2), category(4), datetime64[ns](1), float32(1), int64(1), object(1)
memory usage: 1.3+ GB
